In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'read': 6, 'that': 7, 'ate': 2, 'book': 8, 'The': 0, 'the': 3, 'apple': 4, 'dog': 1, 'Everybody': 5}


In [3]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #print('embeds')
        #print(embeds)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        #print('after embeds')
        #print(lstm_out)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [4]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [25]:
def before_training():
    with torch.no_grad():
        inputs = prepare_sequence(training_data[0][0], word_to_ix)
        tag_scores = model(inputs)
        print('before_training')
        print(tag_scores)
before_training()

embeds
tensor([[-0.4587,  1.2255,  0.9923, -2.1391, -1.1966,  0.9581],
        [-1.5071, -0.4586,  1.2074, -0.1481,  0.0641,  0.4635],
        [ 0.3042, -1.2408,  0.4640, -0.4986, -1.1233, -0.0919],
        [-0.1320, -0.2751, -0.2350,  0.0937, -0.7650,  1.8299],
        [-0.1752,  0.6990, -0.6861,  0.7202,  0.1963,  0.6142]])
before_training
tensor([[-1.4389, -0.8998, -1.0324],
        [-1.3481, -0.9553, -1.0340],
        [-1.3307, -1.0134, -0.9869],
        [-1.3702, -0.9567, -1.0167],
        [-1.3722, -0.9440, -1.0290]])


In [6]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
        print('tag')
        print(tag_scores)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

tag
tensor([[-0.0941, -3.1780, -3.0340],
        [-4.8367, -0.0192, -4.5023],
        [-2.1618, -5.0499, -0.1296],
        [-0.1207, -3.2458, -2.5932],
        [-3.3125, -0.0447, -4.9203]])
tag
tensor([[-4.7970, -0.0229, -4.2388],
        [-3.7623, -4.4898, -0.0351],
        [-0.0561, -3.9073, -3.3692],
        [-4.0855, -0.0239, -4.9842]])
tag
tensor([[-0.0934, -3.1864, -3.0405],
        [-4.8443, -0.0191, -4.5090],
        [-2.1691, -5.0591, -0.1286],
        [-0.1196, -3.2540, -2.6018],
        [-3.3207, -0.0443, -4.9290]])
tag
tensor([[-4.8062, -0.0228, -4.2457],
        [-3.7718, -4.4974, -0.0347],
        [-0.0556, -3.9134, -3.3791],
        [-4.0934, -0.0237, -4.9932]])
tag
tensor([[-0.0926, -3.1948, -3.0471],
        [-4.8518, -0.0189, -4.5156],
        [-2.1763, -5.0683, -0.1276],
        [-0.1186, -3.2622, -2.6104],
        [-3.3288, -0.0440, -4.9376]])
tag
tensor([[-4.8154, -0.0226, -4.2526],
        [-3.7813, -4.5050, -0.0344],
        [-0.0551, -3.9196, -3.3891],
        [

In [36]:
def after_training():
    with torch.no_grad():
        inputs = prepare_sequence(training_data[0][0], word_to_ix)
        tag_scores = model(inputs)
        print('after_training')
        print(tag_scores)


after_training()

embeds
tensor([[-0.0550, -0.5487,  0.1127,  0.9592,  0.9882, -1.6807],
        [ 0.0279,  0.0618, -0.1760, -0.5882, -0.3724, -0.0724],
        [ 1.1192,  0.1657, -2.2605,  0.4327,  0.4711,  1.0474],
        [-0.6625, -0.9285,  0.8376, -0.3564,  1.0423, -1.2305],
        [-0.9383,  0.9925,  0.9525, -1.1533,  0.8349, -0.2281]])
and embeds
tensor([[-0.0550, -0.5487,  0.1127,  0.9592,  0.9882, -1.6807],
        [ 0.0279,  0.0618, -0.1760, -0.5882, -0.3724, -0.0724],
        [ 1.1192,  0.1657, -2.2605,  0.4327,  0.4711,  1.0474],
        [-0.6625, -0.9285,  0.8376, -0.3564,  1.0423, -1.2305],
        [-0.9383,  0.9925,  0.9525, -1.1533,  0.8349, -0.2281]])
after_training
tensor([[-0.0211, -4.2920, -4.9284],
        [-6.0006, -0.0086, -5.1041],
        [-5.1511, -5.2847, -0.0109],
        [-0.0076, -6.2434, -5.1864],
        [-5.9113, -0.0045, -6.3074]])
